In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
mofid = pd.read_csv("../data/main_dataset_mofid_data_V2.csv", index_col=0)
mofid = mofid[["smiles_linkers", "smiles_nodes", "cifname"]].set_index("cifname")
mordred = pd.read_csv("../data/all_f_main_dataset_mordred_V2.csv", index_col=0)
zeo = pd.read_csv("../data/main_dataset_zeo_V2.csv", index_col=0)
cif = pd.read_csv("../data/main_dataset_cif_property_V2.csv", index_col=0)



In [3]:
mordred.shape, zeo.shape, cif.shape

((151, 1826), (162, 3), (165, 8))

In [4]:
def make_mordred():
    from mordred import Calculator, descriptors
    from rdkit import Chem
    mofid = pd.read_csv("../data/main_dataset_mofid_data_V2.csv", index_col=0)
    mofid = mofid[["smiles_linkers", "smiles_nodes", "cifname"]].set_index("cifname")
    linkers = [mofid.smiles_linkers[i].replace("'", "")[1:-1].split(", ") for i in range(mofid.__len__())]
    mols = [[Chem.MolFromSmiles(smi) for smi in smi_list] for smi_list in linkers]
    calc = Calculator(descriptors, ignore_3D=False)
    def f(mof):
        try: return calc.pandas(mof)
        except TypeError:
            return None
    dfs = [f(mof) for mof in mols]
    data_mordred = pd.DataFrame(columns=dfs[0].columns)

    for i, filename in enumerate(mofid.index):
        try:
            if linkers[i] != [""]:
                data_mordred.loc[filename] = dfs[i].mean()
        except AttributeError:
            print(f"{filename:_^20}")
    data_mordred = data_mordred.set_index(data_mordred.index.map(lambda name: name.replace(" ", "")))
    #data_mordred.to_csv("../data/all_f_main_dataset_mordred_V2.csv")
    return data_mordred



In [5]:
def metal_from_node(node: str):
    import re
    """
    input: smilesNodes: str
    return: 
    'metals' in node: list
    'unique' types of metals: np.array, dtype='<U2'
    'count' of unique: np.array, dtype=int
    """
    # "O[Zr]123(O)[OH]4[Zr]56([O]3[Zr]37([OH]2[Zr]28([O]1[Zr]14([O]6[Zr]([OH]53)([OH]21)([O]78)(O)O)([OH2])([OH2])O)[OH2])([OH2])([OH2])O)[OH2]"
    node = node.replace("OH", "").replace("O", "")
    node = node.replace("[", "").replace("]", "").replace(")", "").replace("(", "").replace(",", "")
    node = re.sub(r"\d", "", node) # replace numbers
    #print(node)
    # "ZrZrZrZrZrZr"
    start_cut = 0
    metals = []
    for i, char in enumerate(node[1:]):
        if not char.islower():
            metals.append(node[start_cut:i+1])
            start_cut = i+1
    metals.append(node[start_cut:])
    unique, counts = np.unique(np.array(metals), return_counts=True)
    return metals, unique, counts

In [6]:
elemental_descriptors = pd.read_csv("../data/elemental_descriptors.csv")

In [7]:
node_descriptors = pd.DataFrame(columns=("n_metals", 'n_types_metals', 'Atomic_Number', 'Atomic_Weight', 'Atomic Radius',
       'Mulliken EN', 'polarizability(A^3)', 'electron affinity(kJ/mol)'))

d = ['Atomic_Number', 'Atomic_Weight', 'Atomic Radius',
       'Mulliken EN', 'polarizability(A^3)', 'electron affinity(kJ/mol)']

for filename, node in zip(mofid.index.map(lambda name: name.replace(" ", "").replace(".cif", "")), mofid["smiles_nodes"]):
    try:
       metals, unique, count = metal_from_node(node.replace("'", "").replace(" ", "").replace("%", ""))
       #print(metals)
       n_metals = count.sum()
       n_metals_types = count.__len__()
       df = pd.DataFrame(columns=d, index=[range(count.sum())])
       for metal in metals:
          #print(metal)
          df.loc[len(df)] = elemental_descriptors.loc[elemental_descriptors["Symbol"] == metal].loc[:,d].iloc[0]
       node_descriptors.loc[filename] = n_metals, n_metals_types, *df.mean().array
    except IndexError:
       print(f"Error with {node}")

Error with []
Error with []
Error with []
Error with []
Error with ['*']
Error with ['*']
Error with []
Error with []
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']
Error with ['*']


In [8]:
node_descriptors = node_descriptors.loc[node_descriptors["n_types_metals"] == 1.]


In [9]:
zeo.index = zeo.index.map(lambda s: s.replace(".res", ""))

In [10]:
data_pld_lcd = zeo[["num1", "num2"]].rename({"num1": "lcd", "num2": "pld"}, axis=1)

In [11]:
cif = cif.rename({ "sg_number": "spacegroupNumber"}, axis=1)

In [12]:
cif.index = cif.index.map(lambda name: name.replace(" ", "").replace(".cif", ""))

In [13]:
cif = cif.set_index(cif.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [14]:
node_descriptors = node_descriptors.set_index(node_descriptors.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [15]:
data_pld_lcd = data_pld_lcd.set_index(data_pld_lcd.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [16]:
mordred = mordred.set_index(mordred.index.map(lambda name: name.replace("(", "").replace(")", "")))

In [17]:
data_pld_lcd.shape, cif.shape, mordred.shape, node_descriptors.shape

((162, 2), (165, 8), (151, 1826), (145, 8))

In [18]:
data_main = pd.concat([data_pld_lcd, cif, mordred, node_descriptors], axis=1)

In [19]:
data_main = data_main.loc[list({*list(data_pld_lcd.index)} & {*list(
    cif.index)} & {*list(mordred.index)} & {*list(node_descriptors.index)})]


In [20]:
data_main.shape

(138, 1844)

In [21]:
# data_main.to_csv("../data/main_no_marked.csv")

In [22]:
db_from_to = pd.read_csv("../data/DB_main_with_SG_for_EDA.csv", sep=';')

In [23]:
db_from_to.CIF_init = db_from_to.CIF_init.map(lambda name: name.replace("(", "").replace(")", "").replace(" ", "").replace(".cif", ""))
db_from_to.CIF_final = db_from_to.CIF_final.map(lambda name: name.replace("(", "").replace(")", "").replace(" ", "").replace(".cif", ""))


In [24]:
marked = pd.DataFrame(columns=["target"])

for ind in db_from_to.index:
    if db_from_to.loc[ind, "Stimuli"].find("solvent") != -1 or db_from_to.loc[ind, "Stimuli"].find("gas") != -1 or db_from_to.loc[ind, "Stimuli"].find("humidity") != -1:
        marked.loc[db_from_to.loc[ind, "CIF_init"]] = [1]
        if db_from_to.loc[ind, "Reversible"] == "yes":
            marked.loc[db_from_to.loc[ind, "CIF_final"]] = [1]
        else:
            marked.loc[db_from_to.loc[ind, "CIF_final"]] = [0]
    else:
        marked.loc[db_from_to.loc[ind, "CIF_init"]] = [0]
        marked.loc[db_from_to.loc[ind, "CIF_final"]] = [0]

In [25]:
from pteproc_model import PreprocessingModel
model = joblib.load("../qmof_datasets/scaler.pkl")

In [26]:
data_main.shape

(138, 1844)

In [27]:
main_dataset = data_main[model.cols]

In [28]:
marked_dataset = pd.concat([data_main, marked], axis=1)

In [29]:
marked_dataset = marked_dataset.loc[list({*list(main_dataset.index)} & {*list(marked.index)})]

In [30]:
marked_dataset.shape

(102, 1845)

In [31]:
target, counts = np.unique(marked_dataset.target, return_counts=True)


print(f"|{'target':^10}|{'counts':^10}|")
print(f"{'-':-^23}")
print(f"|{'yes':^10}|{counts[0]:^10}|")
print(f"|{'no':^10}|{counts[1]:^10}|")
print(f"total count:{marked_dataset.__len__()}")

|  target  |  counts  |
-----------------------
|   yes    |    62    |
|    no    |    40    |
total count:102


In [32]:
marked_dataset.shape

(102, 1845)

In [33]:
x, y = model.transform(marked_dataset.drop(['target'], axis=1)), marked_dataset['target']

In [34]:
x.shape, x.dropna().shape

((102, 1018), (102, 1018))

In [35]:
x

,pld,lcd,a,b,c,alpha,beta,gamma,volume,spacegroupNumber,...,Zagreb2,mZagreb1,mZagreb2,n_metals,Atomic_Number,Atomic_Weight,Atomic Radius,Mulliken EN,polarizability(A^3),electron affinity(kJ/mol)
toja054913a_si_004,0.061306,0.035478,0.004061,0.461969,0.277182,0.578143,0.526960,0.441638,0.023942,0.515419,...,0.025232,0.000000,0.011278,0.000000,0.494382,0.436748,0.261111,0.838488,0.056569,0.278014
toCCDC_2065039_JUK-20-chx,0.234377,0.241414,0.376349,0.374762,0.217765,0.637025,0.526960,0.441638,0.106903,0.052863,...,0.354582,0.281080,0.277256,0.000000,0.505618,0.456406,0.238889,0.800687,0.043796,0.000000
tod-flexMOFdried975745,0.039053,0.109046,0.176314,0.326269,0.184724,0.376411,0.373636,0.296054,0.053910,0.004405,...,0.188579,0.252766,0.192982,0.000000,0.280899,0.223951,0.172222,0.824742,0.036496,0.258156
fromDUT-49Co296K890364,0.435653,0.590070,0.899239,0.882823,0.563928,0.256326,0.090555,0.062258,0.516771,0.986784,...,0.164675,0.145509,0.137845,0.090909,0.269663,0.224989,0.188889,0.790378,0.049270,0.190071
fromja500530y_si_003,0.112838,0.115905,0.292269,0.259028,0.138910,0.524035,0.225448,0.373451,0.041505,0.004405,...,0.124834,0.132758,0.112782,0.090909,0.303371,0.252930,0.133333,0.841924,0.029197,0.014184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
toDUT-49Zn100K2014973,0.382893,0.606096,1.280045,1.276188,0.831945,0.578143,0.526960,0.441638,1.931464,0.898678,...,0.467463,0.383274,0.431078,0.090909,0.303371,0.252930,0.133333,0.841924,0.029197,0.014184
1initial924307,0.120228,0.114393,0.128949,0.284110,0.155999,0.677595,0.526960,0.441638,0.027010,0.052863,...,0.119522,0.106507,0.129073,0.000000,0.303371,0.252930,0.133333,0.841924,0.029197,0.014184
from31483716,0.088584,0.072798,0.071471,0.478036,0.288130,0.578143,0.526960,0.441638,0.045042,0.233480,...,0.094290,0.093381,0.105263,0.000000,0.292135,0.244950,0.150000,0.852234,0.034672,0.268085
fromVMOP-+1878068,1.444280,1.581787,1.564889,1.570427,1.032422,0.256326,0.090555,0.062258,2.397011,0.995595,...,0.054892,0.085318,0.052214,0.000000,0.224719,0.190406,0.294444,0.549828,0.138686,0.171631


In [36]:
x.to_csv("../main_datasets/dataset.csv")

In [37]:
y.to_csv("../main_datasets/target.csv")